## 使用 `czsc` 进行选股
---


In [1]:
import sys
sys.path.insert(0, ".")
sys.path.insert(0, "..")

import czsc
print(czsc.__version__)

from datetime import datetime
from typing import List
import traceback
from tqdm import tqdm
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm
from czsc.analyze import KlineAnalyze
# 导入聚宽数据

pd.set_option('display.max_rows', None)



0.5.9


In [2]:
import tushare as ts
# 如果是第一次使用需要设置 token
ts.set_token("565ee6d69fb85cb0bc7fdfc8dea4b8ce1f18366f30e8d23a253bb9cf")
from czsc.data.ts import *


In [2]:
from czsc.data.bs import *

login success!
login respond error_code:0
login respond  error_msg:success


### 笔中枢三买选股
---

条件描述： 最近五笔走势，前三笔构成中枢，第四笔离开中枢，第五笔向下不回中枢



In [3]:
def selector_bi01(kline, res):
    """笔中枢三买选股"""
    
    try:
        ka = KlineAnalyze(kline, bi_mode="old", max_count=1000)
        points = ka.bi_list[-7:]

        if len(points) == 7 and points[-1]['fx_mark'] == "d":
            zs_g = min([x['bi'] for x in points[1:5] if x['fx_mark'] == 'g'])
            zs_d = max([x['bi'] for x in points[1:5] if x['fx_mark'] == 'd'])

            if points[-1]['bi'] > zs_g > zs_d and points[0]['bi'] < zs_d:
                res['symbol'] = ka.symbol
                res['is_match'] = True
                res['dt'] = ka.end_dt
                res['price'] = ka.latest_price
                res['notes'] = ka.fx_list[-1]
                res['fx_h'] = ka.fx_list[-1]["fx_high"]
                res['fx_l'] = ka.fx_list[-1]["fx_low"]
                res['笔中枢三买选股'] = True

    except:
        print("{} 分析失败".format(symbol))
        traceback.print_exc()
    return res

### 三笔回调构成三买
---

In [4]:
def selector_bi02(kline, res):
    """三笔回调构成三买"""

    try:
        ka = KlineAnalyze(kline, bi_mode="old", max_count=1000)
        if len(ka.bi_list) > 16 and ka.bi_list[-1]['fx_mark'] == 'd':
            points = ka.bi_list[-16:]
            fd1_points = [x['bi'] for x in points[0: 4]]
            fd2_points = [x['bi'] for x in points[3: 7]]
            fd3_points = [x['bi'] for x in points[6: 10]]
            fd4_points = [x['bi'] for x in points[9: 13]]
            fd5_points = [x['bi'] for x in points[12: 16]]
            fd1 = {"high": max(fd1_points), "low": min(fd1_points)}
            fd2 = {"high": max(fd2_points), "low": min(fd2_points)}
            fd3 = {"high": max(fd3_points), "low": min(fd3_points)}
            fd4 = {"high": max(fd4_points), "low": min(fd4_points)}
            fd5 = {"high": max(fd5_points), "low": min(fd5_points)}

            zs_g = min(fd1['high'], fd2['high'], fd3['high'])
            zs_d = max(fd1['low'], fd2['low'], fd3['low'])
            if zs_d < zs_g < fd5['low'] < fd4['high'] and fd4['high'] > max(fd1['high'], fd3['high']):
                res['symbol'] = ka.symbol
                res['is_match'] = True
                res['dt'] = ka.end_dt
                res['price'] = ka.latest_price
                res['notes'] = ka.fx_list[-1]
                res['fx_h'] = ka.fx_list[-1]["fx_high"]
                res['fx_l'] = ka.fx_list[-1]["fx_low"]
                res['三笔回调构成三买'] = True
    except:
        print(ka.fx_list)
        traceback.print_exc()
    return res

## 执行选股
---

In [11]:

# 如果是第一次使用需要设置 token
# set_token("手机号", "密码")

# 使用聚宽数据在创业板综指上选股
# symbols = get_index_stocks("399006.XSHE")
symbols = get_index_stocks("zz500")
freq = "d"

results = []
for symbol in tqdm(symbols, desc="selector"):
    res = {"is_match": False, "notes": None, "dt": None, "price": None, "fx_h": None, "fx_l": None,
           "name": "selector_bi02", "symbol": None, "三笔回调构成三买": False, "笔中枢三买选股": False, "name": None}
    try:
        kline = get_kline(symbol, end_date=datetime.now(), freq=freq, count=200)
        for selector in [selector_bi01, selector_bi02]:
            try:
                res = selector(kline, res)
            except:
                continue
        if res['is_match']:
            res['name'] = get_stock_info(symbol).code_name[0]
            results.append(res)
    except:
        traceback.print_exc()
        print("fail on: {}".format(symbol))

print("选股结果：")
zz500df = pd.DataFrame(results)
# df.to_excel("选股结果.xlsx", index=False)

<ipython-input-11-c6d040aabcc4>:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for symbol in tqdm(symbols, desc="selector"):


selector:   0%|          | 0/500 [00:00<?, ?it/s]

选股结果：


In [15]:

# 如果是第一次使用需要设置 token
# set_token("手机号", "密码")

# 使用聚宽数据在创业板综指上选股
# symbols = get_index_stocks("399006.XSHE")
symbols = get_index_stocks("all")
freq = "d"

results = []
for symbol in tqdm(symbols, desc="selector"):
    res = {"is_match": False, "notes": None, "dt": None, "price": None, "fx_h": None, "fx_l": None,
           "name": "selector_bi02", "symbol": None, "三笔回调构成三买": False, "笔中枢三买选股": False, "name": None}
    try:
        kline = get_kline(symbol, end_date=datetime.now(), freq=freq, count=200)
        for selector in [selector_bi01, selector_bi02]:
            try:
                res = selector(kline, res)
            except:
                continue
        if res['is_match']:
            res['name'] = get_stock_info(symbol).code_name[0]
            results.append(res)
    except:
        traceback.print_exc()
        print("fail on: {}".format(symbol))

print("选股结果：")
alldf = pd.DataFrame(results)
# df.to_excel("选股结果.xlsx", index=False)


<ipython-input-15-db74f73a3d74>:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for symbol in tqdm(symbols, desc="selector"):


selector:   0%|          | 0/4232 [00:00<?, ?it/s]

[Errno 104] Connection reset by peer
接收数据异常，请稍后再试。
[Errno 32] Broken pipe
接收数据异常，请稍后再试。
fail on: sh.600662
[Errno 32] Broken pipe
接收数据异常，请稍后再试。
fail on: sz.002337
[Errno 32] Broken pipe
接收数据异常，请稍后再试。
fail on: sh.603389
[Errno 32] Broken pipe
接收数据异常，请稍后再试。
fail on: sh.601016
[Errno 32] Broken pipe
接收数据异常，请稍后再试。
fail on: sz.002421
[Errno 32] Broken pipe
接收数据异常，请稍后再试。
fail on: sz.002065
[Errno 32] Broken pipe
接收数据异常，请稍后再试。
fail on: sz.002637
[Errno 32] Broken pipe
接收数据异常，请稍后再试。
fail on: sh.688312
[Errno 32] Broken pipe
接收数据异常，请稍后再试。
fail on: sz.002997
[Errno 32] Broken pipe
接收数据异常，请稍后再试。
fail on: sz.300501
[Errno 32] Broken pipe
接收数据异常，请稍后再试。
fail on: sz.000916
[Errno 32] Broken pipe
接收数据异常，请稍后再试。
fail on: sz.300222
[Errno 32] Broken pipe
接收数据异常，请稍后再试。
fail on: sz.002353
[Errno 32] Broken pipe
接收数据异常，请稍后再试。
fail on: sz.000851
[Errno 32] Broken pipe
接收数据异常，请稍后再试。
fail on: sz.002462
[Errno 32] Broken pipe
接收数据异常，请稍后再试。
fail on: sh.688500
[Errno 32] Broken pipe
接收数据异常，请稍后再试。
fail on: sh.6039

Traceback (most recent call last):
  File "<ipython-input-15-db74f73a3d74>", line 14, in <module>
    kline = get_kline(symbol, end_date=datetime.now(), freq=freq, count=200)
  File "../czsc/data/bs.py", line 131, in get_kline
    df.sort_values('dt', inplace=True)
  File "/usr/local/lib/python3.8/site-packages/pandas/core/frame.py", line 5008, in sort_values
    k = self._get_label_or_level_values(by, axis=axis)
  File "/usr/local/lib/python3.8/site-packages/pandas/core/generic.py", line 1774, in _get_label_or_level_values
    raise KeyError(key)
KeyError: 'dt'
Traceback (most recent call last):
  File "<ipython-input-15-db74f73a3d74>", line 14, in <module>
    kline = get_kline(symbol, end_date=datetime.now(), freq=freq, count=200)
  File "../czsc/data/bs.py", line 131, in get_kline
    df.sort_values('dt', inplace=True)
  File "/usr/local/lib/python3.8/site-packages/pandas/core/frame.py", line 5008, in sort_values
    k = self._get_label_or_level_values(by, axis=axis)
  File "/usr/l

fail on: sz.002682
[Errno 32] Broken pipe
接收数据异常，请稍后再试。
fail on: sh.600887
[Errno 32] Broken pipe
接收数据异常，请稍后再试。
fail on: sz.001896
[Errno 32] Broken pipe
接收数据异常，请稍后再试。
fail on: sz.002790
[Errno 32] Broken pipe
接收数据异常，请稍后再试。
fail on: sz.002871
[Errno 32] Broken pipe
接收数据异常，请稍后再试。
fail on: sh.600143
[Errno 32] Broken pipe
接收数据异常，请稍后再试。
fail on: sh.603259
[Errno 32] Broken pipe
接收数据异常，请稍后再试。
fail on: sh.603077
[Errno 32] Broken pipe
接收数据异常，请稍后再试。
fail on: sz.002397
[Errno 32] Broken pipe
接收数据异常，请稍后再试。
fail on: sz.002662
[Errno 32] Broken pipe
接收数据异常，请稍后再试。
fail on: sz.002987
[Errno 32] Broken pipe
接收数据异常，请稍后再试。
fail on: sz.300079
[Errno 32] Broken pipe
接收数据异常，请稍后再试。
fail on: sz.300107
[Errno 32] Broken pipe
接收数据异常，请稍后再试。
fail on: sz.300166
[Errno 32] Broken pipe
接收数据异常，请稍后再试。
fail on: sz.300464
[Errno 32] Broken pipe
接收数据异常，请稍后再试。
fail on: sh.600185
[Errno 32] Broken pipe
接收数据异常，请稍后再试。
fail on: sh.600325
[Errno 32] Broken pipe
接收数据异常，请稍后再试。
fail on: sh.688139
[Errno 32] Broken pipe
接收数据异常

...

Traceback (most recent call last):
  File "<ipython-input-15-db74f73a3d74>", line 14, in <module>
    kline = get_kline(symbol, end_date=datetime.now(), freq=freq, count=200)
  File "../czsc/data/bs.py", line 131, in get_kline
    df.sort_values('dt', inplace=True)
  File "/usr/local/lib/python3.8/site-packages/pandas/core/frame.py", line 5008, in sort_values
    k = self._get_label_or_level_values(by, axis=axis)
  File "/usr/local/lib/python3.8/site-packages/pandas/core/generic.py", line 1774, in _get_label_or_level_values
    raise KeyError(key)
KeyError: 'dt'
Traceback (most recent call last):
  File "<ipython-input-15-db74f73a3d74>", line 14, in <module>
    kline = get_kline(symbol, end_date=datetime.now(), freq=freq, count=200)
  File "../czsc/data/bs.py", line 131, in get_kline
    df.sort_values('dt', inplace=True)
  File "/usr/local/lib/python3.8/site-packages/pandas/core/frame.py", line 5008, in sort_values
    k = self._get_label_or_level_values(by, axis=axis)
  File "/usr/l

[Errno 32] Broken pipe
接收数据异常，请稍后再试。
fail on: sz.002990
[Errno 32] Broken pipe
接收数据异常，请稍后再试。
fail on: sh.600480
[Errno 32] Broken pipe
接收数据异常，请稍后再试。
fail on: sh.601318
[Errno 32] Broken pipe
接收数据异常，请稍后再试。
fail on: sh.603330
[Errno 32] Broken pipe
接收数据异常，请稍后再试。
fail on: sz.000622
[Errno 32] Broken pipe
接收数据异常，请稍后再试。
fail on: sh.600984
[Errno 32] Broken pipe
接收数据异常，请稍后再试。
fail on: sz.002396
[Errno 32] Broken pipe
接收数据异常，请稍后再试。
fail on: sz.002632
[Errno 32] Broken pipe
接收数据异常，请稍后再试。
fail on: sz.002892
[Errno 32] Broken pipe
接收数据异常，请稍后再试。
fail on: sz.300061
[Errno 32] Broken pipe
接收数据异常，请稍后再试。
fail on: sh.605003
[Errno 32] Broken pipe
接收数据异常，请稍后再试。
fail on: sz.300124
[Errno 32] Broken pipe
接收数据异常，请稍后再试。
fail on: sz.300224
[Errno 32] Broken pipe
接收数据异常，请稍后再试。
fail on: sz.300527
[Errno 32] Broken pipe
接收数据异常，请稍后再试。
fail on: sh.600435
[Errno 32] Broken pipe
接收数据异常，请稍后再试。
fail on: sh.603008
[Errno 32] Broken pipe
接收数据异常，请稍后再试。
fail on: sh.600022
[Errno 32] Broken pipe
接收数据异常，请稍后再试。
fail on: sh

  File "/usr/local/lib/python3.8/site-packages/pandas/core/generic.py", line 1774, in _get_label_or_level_values
    raise KeyError(key)
KeyError: 'dt'
Traceback (most recent call last):
  File "<ipython-input-15-db74f73a3d74>", line 14, in <module>
    kline = get_kline(symbol, end_date=datetime.now(), freq=freq, count=200)
  File "../czsc/data/bs.py", line 131, in get_kline
    df.sort_values('dt', inplace=True)
  File "/usr/local/lib/python3.8/site-packages/pandas/core/frame.py", line 5008, in sort_values
    k = self._get_label_or_level_values(by, axis=axis)
  File "/usr/local/lib/python3.8/site-packages/pandas/core/generic.py", line 1774, in _get_label_or_level_values
    raise KeyError(key)
KeyError: 'dt'
Traceback (most recent call last):
  File "<ipython-input-15-db74f73a3d74>", line 14, in <module>
    kline = get_kline(symbol, end_date=datetime.now(), freq=freq, count=200)
  File "../czsc/data/bs.py", line 131, in get_kline
    df.sort_values('dt', inplace=True)
  File "/usr/

fail on: sh.603912
[Errno 32] Broken pipe
接收数据异常，请稍后再试。
fail on: sz.300457
[Errno 32] Broken pipe
接收数据异常，请稍后再试。
fail on: sh.600137
[Errno 32] Broken pipe
接收数据异常，请稍后再试。
fail on: sz.002284
[Errno 32] Broken pipe
接收数据异常，请稍后再试。
fail on: sh.601689
[Errno 32] Broken pipe
接收数据异常，请稍后再试。
fail on: sz.300264
[Errno 32] Broken pipe
接收数据异常，请稍后再试。
fail on: sz.002507
[Errno 32] Broken pipe
接收数据异常，请稍后再试。
fail on: sh.601069
[Errno 32] Broken pipe
接收数据异常，请稍后再试。
fail on: sh.688528
[Errno 32] Broken pipe
接收数据异常，请稍后再试。
fail on: sz.002412
[Errno 32] Broken pipe
接收数据异常，请稍后再试。
fail on: sz.002707
[Errno 32] Broken pipe
接收数据异常，请稍后再试。
fail on: sz.300915
[Errno 32] Broken pipe
接收数据异常，请稍后再试。
fail on: sz.002534
[Errno 32] Broken pipe
接收数据异常，请稍后再试。
fail on: sz.002769
[Errno 32] Broken pipe
接收数据异常，请稍后再试。
fail on: sz.000407
[Errno 32] Broken pipe
接收数据异常，请稍后再试。
fail on: sh.600060
[Errno 32] Broken pipe
接收数据异常，请稍后再试。
fail on: sz.000893
[Errno 32] Broken pipe
接收数据异常，请稍后再试。
fail on: sh.600881
[Errno 32] Broken pipe
接收数据异常

KeyError: 'dt'
Traceback (most recent call last):
  File "<ipython-input-15-db74f73a3d74>", line 14, in <module>
    kline = get_kline(symbol, end_date=datetime.now(), freq=freq, count=200)
  File "../czsc/data/bs.py", line 131, in get_kline
    df.sort_values('dt', inplace=True)
  File "/usr/local/lib/python3.8/site-packages/pandas/core/frame.py", line 5008, in sort_values
    k = self._get_label_or_level_values(by, axis=axis)
  File "/usr/local/lib/python3.8/site-packages/pandas/core/generic.py", line 1774, in _get_label_or_level_values
    raise KeyError(key)
KeyError: 'dt'
Traceback (most recent call last):
  File "<ipython-input-15-db74f73a3d74>", line 14, in <module>
    kline = get_kline(symbol, end_date=datetime.now(), freq=freq, count=200)
  File "../czsc/data/bs.py", line 131, in get_kline
    df.sort_values('dt', inplace=True)
  File "/usr/local/lib/python3.8/site-packages/pandas/core/frame.py", line 5008, in sort_values
    k = self._get_label_or_level_values(by, axis=axis)

In [5]:

# 如果是第一次使用需要设置 token
# set_token("手机号", "密码")

# 使用聚宽数据在创业板综指上选股
# symbols = get_index_stocks("399006.XSHE")
symbols = get_index_stocks("hs300")
freq = "d"

results = []
for symbol in tqdm(symbols, desc="selector"):
    res = {"is_match": False, "notes": None, "dt": None, "price": None, "fx_h": None, "fx_l": None,
           "name": "selector_bi02", "symbol": None, "三笔回调构成三买": False, "笔中枢三买选股": False, "name": None}
    try:
        kline = get_kline(symbol, end_date=datetime.now(), freq=freq, count=200)
        for selector in [selector_bi01, selector_bi02]:
            try:
                res = selector(kline, res)
            except:
                continue
        if res['is_match']:
            res['name'] = get_stock_info(symbol).code_name[0]
            results.append(res)
    except:
        traceback.print_exc()
        print("fail on: {}".format(symbol))

print("选股结果：")
hs300df = pd.DataFrame(results)
# df.to_excel("选股结果.xlsx", index=False)

<ipython-input-5-06ce22f98b04>:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for symbol in tqdm(symbols, desc="selector"):


selector:   0%|          | 0/300 [00:00<?, ?it/s]

../czsc/data/bs.py:151: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  k[col] = k[col].apply(round, args=(2,))


选股结果：


In [6]:
pd.set_option('display.max_columns', None)
hs300df

,is_match,notes,dt,price,fx_h,fx_l,name,symbol,三笔回调构成三买,笔中枢三买选股
0,True,"{'dt': 2020-12-28 00:00:00, 'fx_mark': 'd', 'f...",2021-01-06,64.07,61.13,58.69,格力电器,sz.000651,False,True
1,True,"{'dt': 2020-12-31 00:00:00, 'fx_mark': 'g', 'f...",2021-01-06,21.13,22.45,21.26,长安汽车,sz.000625,True,False
2,True,"{'dt': 2020-12-29 00:00:00, 'fx_mark': 'g', 'f...",2021-01-06,22.97,26.25,23.39,上汽集团,sh.600104,True,False
3,True,"{'dt': 2020-12-29 00:00:00, 'fx_mark': 'd', 'f...",2021-01-06,42.05,38.78,36.63,科大讯飞,sz.002230,True,False
4,True,"{'dt': 2020-12-31 00:00:00, 'fx_mark': 'd', 'f...",2021-01-06,21.07,20.83,19.54,江西铜业,sh.600362,True,False
5,True,"{'dt': 2020-12-29 00:00:00, 'fx_mark': 'd', 'f...",2021-01-06,294.45,252.70,243.30,片仔癀,sh.600436,False,True
6,True,"{'dt': 2020-12-25 00:00:00, 'fx_mark': 'd', 'f...",2021-01-06,38.24,35.35,31.79,中航西飞,sz.000768,False,True
7,True,"{'dt': 2020-12-25 00:00:00, 'fx_mark': 'd', 'f...",2021-01-06,10.73,9.46,9.06,中联重科,sz.000157,False,True
8,True,"{'dt': 2020-12-22 00:00:00, 'fx_mark': 'd', 'f...",2021-01-06,73.91,69.50,65.88,亿联网络,sz.300628,False,True
9,True,"{'dt': 2020-12-29 00:00:00, 'fx_mark': 'd', 'f...",2021-01-06,4.67,4.24,4.12,中航资本,sh.600705,True,False


In [12]:
pd.set_option('display.max_rows', 1000)
zz500df

,is_match,notes,dt,price,fx_h,fx_l,name,symbol,三笔回调构成三买,笔中枢三买选股
0,True,"{'dt': 2021-01-05 00:00:00, 'fx_mark': 'd', 'f...",2021-01-06,16.19,15.88,14.67,中集集团,sz.000039,False,True
1,True,"{'dt': 2020-12-28 00:00:00, 'fx_mark': 'd', 'f...",2021-01-06,61.22,54.18,52.30,洽洽食品,sz.002557,True,False
2,True,"{'dt': 2020-12-30 00:00:00, 'fx_mark': 'd', 'f...",2021-01-06,20.79,20.19,19.10,国联证券,sh.601456,False,True
3,True,"{'dt': 2020-12-31 00:00:00, 'fx_mark': 'd', 'f...",2021-01-06,38.90,36.23,34.02,迎驾贡酒,sh.603198,False,True
4,True,"{'dt': 2021-01-05 00:00:00, 'fx_mark': 'g', 'f...",2021-01-06,7.16,7.33,7.03,平高电气,sh.600312,False,True
5,True,"{'dt': 2020-12-30 00:00:00, 'fx_mark': 'd', 'f...",2021-01-06,6.07,6.33,6.02,山东高速,sh.600350,False,True
6,True,"{'dt': 2021-01-04 00:00:00, 'fx_mark': 'g', 'f...",2021-01-06,8.33,8.88,8.20,燕京啤酒,sz.000729,True,False
7,True,"{'dt': 2020-12-29 00:00:00, 'fx_mark': 'd', 'f...",2021-01-06,74.63,66.90,63.40,口子窖,sh.603589,True,False
8,True,"{'dt': 2021-01-04 00:00:00, 'fx_mark': 'g', 'f...",2021-01-06,12.26,14.30,12.75,国盛金控,sz.002670,False,True
9,True,"{'dt': 2020-12-31 00:00:00, 'fx_mark': 'd', 'f...",2021-01-06,29.33,27.23,26.46,圣农发展,sz.002299,False,True


In [7]:
hs300df.sort_values('symbol', inplace=True)

In [8]:
hs300df

,is_match,notes,dt,price,fx_h,fx_l,name,symbol,三笔回调构成三买,笔中枢三买选股
2,True,"{'dt': 2020-12-29 00:00:00, 'fx_mark': 'g', 'f...",2021-01-06,22.97,26.25,23.39,上汽集团,sh.600104,True,False
4,True,"{'dt': 2020-12-31 00:00:00, 'fx_mark': 'd', 'f...",2021-01-06,21.07,20.83,19.54,江西铜业,sh.600362,True,False
5,True,"{'dt': 2020-12-29 00:00:00, 'fx_mark': 'd', 'f...",2021-01-06,294.45,252.70,243.30,片仔癀,sh.600436,False,True
16,True,"{'dt': 2020-12-29 00:00:00, 'fx_mark': 'd', 'f...",2021-01-06,108.95,100.00,95.10,青岛啤酒,sh.600600,False,True
9,True,"{'dt': 2020-12-29 00:00:00, 'fx_mark': 'd', 'f...",2021-01-06,4.67,4.24,4.12,中航资本,sh.600705,True,False
13,True,"{'dt': 2020-12-31 00:00:00, 'fx_mark': 'd', 'f...",2021-01-06,3.73,3.81,3.53,中国铝业,sh.601600,False,True
15,True,"{'dt': 2021-01-05 00:00:00, 'fx_mark': 'd', 'f...",2021-01-06,5.01,4.95,4.75,中国核电,sh.601985,True,True
14,True,"{'dt': 2020-12-30 00:00:00, 'fx_mark': 'd', 'f...",2021-01-06,206.62,201.00,194.60,海天味业,sh.603288,False,True
17,True,"{'dt': 2020-12-24 00:00:00, 'fx_mark': 'd', 'f...",2021-01-06,159.71,142.45,130.60,传音控股,sh.688036,False,True
7,True,"{'dt': 2020-12-25 00:00:00, 'fx_mark': 'd', 'f...",2021-01-06,10.73,9.46,9.06,中联重科,sz.000157,False,True


In [9]:
hs300df[(hs300df['三笔回调构成三买'] == False) & (hs300df['笔中枢三买选股'] == True)]

,is_match,notes,dt,price,fx_h,fx_l,name,symbol,三笔回调构成三买,笔中枢三买选股
5,True,"{'dt': 2020-12-29 00:00:00, 'fx_mark': 'd', 'f...",2021-01-06,294.45,252.70,243.30,片仔癀,sh.600436,False,True
16,True,"{'dt': 2020-12-29 00:00:00, 'fx_mark': 'd', 'f...",2021-01-06,108.95,100.00,95.10,青岛啤酒,sh.600600,False,True
13,True,"{'dt': 2020-12-31 00:00:00, 'fx_mark': 'd', 'f...",2021-01-06,3.73,3.81,3.53,中国铝业,sh.601600,False,True
14,True,"{'dt': 2020-12-30 00:00:00, 'fx_mark': 'd', 'f...",2021-01-06,206.62,201.00,194.60,海天味业,sh.603288,False,True
17,True,"{'dt': 2020-12-24 00:00:00, 'fx_mark': 'd', 'f...",2021-01-06,159.71,142.45,130.60,传音控股,sh.688036,False,True
7,True,"{'dt': 2020-12-25 00:00:00, 'fx_mark': 'd', 'f...",2021-01-06,10.73,9.46,9.06,中联重科,sz.000157,False,True
12,True,"{'dt': 2020-12-30 00:00:00, 'fx_mark': 'd', 'f...",2021-01-06,136.56,115.59,109.28,云南白药,sz.000538,False,True
0,True,"{'dt': 2020-12-28 00:00:00, 'fx_mark': 'd', 'f...",2021-01-06,64.07,61.13,58.69,格力电器,sz.000651,False,True
6,True,"{'dt': 2020-12-25 00:00:00, 'fx_mark': 'd', 'f...",2021-01-06,38.24,35.35,31.79,中航西飞,sz.000768,False,True
8,True,"{'dt': 2020-12-22 00:00:00, 'fx_mark': 'd', 'f...",2021-01-06,73.91,69.50,65.88,亿联网络,sz.300628,False,True


In [10]:
hs300df[(hs300df['三笔回调构成三买'] == True) & (hs300df['笔中枢三买选股'] == True)]

,is_match,notes,dt,price,fx_h,fx_l,name,symbol,三笔回调构成三买,笔中枢三买选股
15,True,"{'dt': 2021-01-05 00:00:00, 'fx_mark': 'd', 'f...",2021-01-06,5.01,4.95,4.75,中国核电,sh.601985,True,True


In [13]:
zz500df.sort_values('symbol', inplace=True)

In [14]:
zz500df[(zz500df['三笔回调构成三买'] == True) & (zz500df['笔中枢三买选股'] == True)]

,is_match,notes,dt,price,fx_h,fx_l,name,symbol,三笔回调构成三买,笔中枢三买选股
15,True,"{'dt': 2020-12-31 00:00:00, 'fx_mark': 'd', 'f...",2021-01-06,28.67,26.63,25.6,卫星石化,sz.002648,True,True


In [16]:
alldf.sort_values('symbol', inplace=True)

In [17]:
alldf[(alldf['三笔回调构成三买'] == True) & (alldf['笔中枢三买选股'] == True)]

,is_match,notes,dt,price,fx_h,fx_l,name,symbol,三笔回调构成三买,笔中枢三买选股
44,True,"{'dt': 2021-01-04 00:00:00, 'fx_mark': 'g', 'f...",2021-01-06,49.29,57.20,50.51,容百科技,sh.688005,True,True
105,True,"{'dt': 2021-01-04 00:00:00, 'fx_mark': 'd', 'f...",2021-01-06,9.82,9.45,9.09,绿景控股,sz.000502,True,True
109,True,"{'dt': 2020-12-31 00:00:00, 'fx_mark': 'd', 'f...",2021-01-06,28.67,26.63,25.60,卫星石化,sz.002648,True,True
26,True,"{'dt': 2021-01-05 00:00:00, 'fx_mark': 'g', 'f...",2021-01-06,5.62,7.08,6.21,浙江世宝,sz.002703,True,True
104,True,"{'dt': 2021-01-04 00:00:00, 'fx_mark': 'g', 'f...",2021-01-06,7.47,8.88,7.81,珈伟新能,sz.300317,True,True
58,True,"{'dt': 2021-01-04 00:00:00, 'fx_mark': 'g', 'f...",2021-01-06,7.86,8.45,8.07,大烨智能,sz.300670,True,True
87,True,"{'dt': 2020-12-30 00:00:00, 'fx_mark': 'd', 'f...",2021-01-06,51.24,37.84,34.88,隆利科技,sz.300752,True,True
